In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections

import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile
import gensim
import scipy.sparse
import string

import numpy as np
import pandas as pd
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from scipy.sparse import coo_matrix
from scipy import sparse

from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.utils.extmath import randomized_svd

from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import svds, eigs
from numpy.linalg import solve, norm
from numpy.random import rand
from gensim.models import Word2Vec
from collections import Counter
from math import log

import itertools
from gensim.models.word2vec import Text8Corpus
from glove import Glove
from glove import Corpus

rng = np.random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
config.gpu_options.allow_growth = True #allocate dynamically 
sess = tf.Session(config = config)

In [2]:
def maybe_download(filename, url, path):
    """Download a file if not present, and make sure it's the right size."""
    local_filename = os.path.join(path, filename)
    if not os.path.exists(local_filename):      
        local_filename, _ = urllib.request.urlretrieve(url + filename,
                                                   local_filename)
    statinfo = os.stat(local_filename)
    return local_filename

# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0]))
    return data


filename = maybe_download('enwik9.zip', 'http://mattmahoney.net/dc/', '/home/aidana/')
vocabulary = read_data(filename)
      
words = vocabulary.split('.')

# convert to lower case

words = [word.lower() for word in words]


In [3]:
we = []
for i in words:
    i = i.split()
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in i]

    # remove remaining tokens that are not alphabetic
    sen = [word for word in stripped if word.isalpha()]
    
    we.append(sen)

In [4]:
from gensim.models import Word2Vec
model = Word2Vec(we, size=300, window=2, min_count= 50, workers=4, sg = 1, negative=5)
#model = Word2Vec.load('text8_gensim') #load saved model

In [5]:
model.train(we, total_examples=model.corpus_count, epochs = 15) #train a model

(1298288921, 1780761075)

In [6]:
model.save('Word2Vec_enwik_300d') # save model

In [ ]:
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
print(test_data_dir)
model.evaluate_word_pairs(test_data_dir +'wordsim353.tsv')

In [ ]:
acc = model.accuracy(test_data_dir + 'questions-words.txt')
corr_num = sum([len(section["correct"]) for section in acc])
incorr_num = sum([len(section["incorrect"]) for section in acc])
print(corr_num/(corr_num+incorr_num))